In [1]:
!pip install langchain pymysql openai python-dotenv


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from llama_index import (
    GPTVectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    GPTSQLStructStoreIndex,
    SQLDatabase,
    WikipediaReader
)

In [2]:
# define pinecone index

from dotenv import load_dotenv
import pinecone
import os

username = str(os.getenv('username'))
passwd = str(os.getenv('passwd'))
hostname = str(os.getenv('hostname'))
name = str(os.getenv('name'))

OPENAI_API_KEY = str(os.getenv('OPENAI_API_KEY'))
PINECONE_API_KEY = str(os.getenv('PINECONE_API_KEY'))
PINECONE_API_ENV = str(os.getenv('PINECONE_API_ENV'))

pinecone.init(api_key=PINECONE_API_KEY, environment='us-west4-gcp-free')

# dimensions are for text-embedding-ada-002
#pinecone.create_index("quickstart", dimension=1536, metric="euclidean", pod_type="p1")

pinecone_index = pinecone.Index("quickstart")

h:\Projects\Large Langugage Models\llm\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [30]:
from llama_index.node_parser.simple import SimpleNodeParser
from llama_index import Document
from llama_index import ServiceContext, LLMPredictor
from llama_index.storage import StorageContext
from llama_index.vector_stores import PineconeVectorStore
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from langchain.chat_models import ChatOpenAI


# define node parser and LLM
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo', streaming=True))
service_context = ServiceContext.from_defaults(chunk_size=1024, llm_predictor=llm_predictor)
text_splitter = TokenTextSplitter(chunk_size=1024)
node_parser = SimpleNodeParser(text_splitter=text_splitter)

# define pinecone vector index
vector_store = PineconeVectorStore(pinecone_index=pinecone_index, namespace='bankdata')
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_index = GPTVectorStoreIndex([], storage_context=storage_context)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
> [build_index_from_nodes] Total embedding token usage: 0 tokens


In [23]:
sql_database = SQLDatabase.from_uri(f"mysql+pymysql://{username}:{passwd}@{hostname}/{name}")

In [24]:
sql_index = GPTSQLStructStoreIndex.from_documents(
    [], 
    sql_database=sql_database
)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
> [build_index_from_nodes] Total embedding token usage: 0 tokens


In [77]:
bank_doc = ["""If you are from Los Angeles, then you have to pay 1% additional as interest on loans and credit. 
This interest is called the richness index. State of california is also called the orange state"""]

In [78]:
documents = [Document(t) for t in bank_doc]

In [79]:
documents

[Document(text='If you are from Los Angeles, then you have to pay 1% additional as interest on loans and credit. \nThis interest is called the richness index. State of california is also called the orange state', doc_id='cdef3cc9-3843-44e6-ae92-b5fac2043da9', embedding=None, doc_hash='ae8a0acaa3491a32d69ed449addc16d6763561392a4e69d55dd62c8fc9c12fb6', extra_info=None)]

In [80]:
for doc in documents:
    nodes = node_parser.get_nodes_from_documents([doc])
    vector_index.insert_nodes(nodes)

INFO:llama_index.token_counter.token_counter:> [insert] Total LLM token usage: 0 tokens
> [insert] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [insert] Total embedding token usage: 43 tokens
> [insert] Total embedding token usage: 43 tokens


In [81]:
from llama_index.query_engine import SQLAutoVectorQueryEngine, RetrieverQueryEngine
from llama_index.tools.query_engine import QueryEngineTool
from llama_index.indices.vector_store import VectorIndexAutoRetriever

In [82]:
sql_query_engine = sql_index.as_query_engine(synthesize_response=True)

In [83]:
from llama_index.vector_stores.types import MetadataInfo, VectorStoreInfo
from llama_index.query_engine.retriever_query_engine import RetrieverQueryEngine

vector_store_info = VectorStoreInfo(content_info='Rules and information about the bank', metadata_info=[])
vector_auto_retriever = VectorIndexAutoRetriever(vector_index, vector_store_info=vector_store_info)
retriever_query_engine = RetrieverQueryEngine.from_args(vector_auto_retriever, service_context=service_context)

In [84]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        'Useful for translating a natural language query into a SQL query over a table containing: '
    )
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=retriever_query_engine,
    description=(
        'Useful for answering semantic questions about the bank'
    )
)

In [85]:
query_engine = SQLAutoVectorQueryEngine(sql_tool, vector_tool, service_context=service_context)

In [92]:
response = query_engine.query('Give me all the customers who have taken a loan')

Querying SQL database: Useful for translating a natural language query into a SQL query over a table containing
INFO:llama_index.query_engine.sql_vector_query_engine:> Querying SQL database: Useful for translating a natural language query into a SQL query over a table containing
> Querying SQL database: Useful for translating a natural language query into a SQL query over a table containing
INFO:llama_index.indices.struct_store.sql_query:> Table desc str: Schema of table Loans:
Table 'Loans' has columns: LoanID (INTEGER), AccountNumber (INTEGER), LoanAmount (DECIMAL(10, 2)), InterestRate (DECIMAL(5, 2)), LoanDate (DATE) and foreign keys: ['AccountNumber'] -> Accounts.['AccountNumber'], ['AccountNumber'] -> Accounts.['AccountNumber'].


Schema of table Checks:
Table 'Checks' has columns: CheckID (INTEGER), AccountNumber (INTEGER), CheckNumber (VARCHAR(20)), Amount (DECIMAL(10, 2)), CheckDate (DATE) and foreign keys: ['AccountNumber'] -> Accounts.['AccountNumber'], ['AccountNumber'] -> A